# Twitter Sentiment Analysis
En este notebook vas a ver un ejemplo de los procesos necesarios para realizar un análisis de sentimientos sobre Tweets. Para ello tendremos que seguir los siguientes pasos:
1. Conseguir un Corpus: no es más que una base de datos de texto etiquetado
2. Limpiar los datos
3. Entrenar un modelo con el corpus
4. Atacar a la API de Twitter
5. Predecir los nuevos Tweets

**Estos programas son muy útiles en campañas de marketing, para monitorizar el lanzamiento de un nuevo producto, realizar seguimiento en Twitter de eventos, o simplemente tener monitorizadas ciertas cuentas o hashtags para tener un programa de análisis real time.**

## 1. Corpus
Para conseguir el corpus tendremos que registrarnos en la [página del TASS](http://tass.sepln.org/tass_data/download.php), que se trata de una asociación de análisis semántico que encargada de recopilar texto y mantenerlo etiquetado. 

Para datasets en ingles lo tenemos más fácil ya que con librerías como [TextBlob](https://textblob.readthedocs.io/en/dev/) podemos predecir directamente la polaridad del Tweet, con modelos ya preentrenados. En el caso del castellano necesitamos acudir a un corpus etiquetado para entrenar nuestro modelo.

Registrate en el TASS y accede a sus corpus a través de un link que te llegará al correo tras el registro.

![imagen](img/tass_register.png)


Una vez estes registrado, descárgate el corpus de tweets en español de entrenamiento. En este punto lo ideal es coger un corpus que se adapte lo máximo posible a los tipos de tweets que intentamos predecir, es decir, si queremos predecir tweets sobre política, procurar elegir un corpus que tenga vocabulario de política.

En este notebook se va a elegir un corpus genérico con no demasiados registros para aligerar la limpieza y entrenamiento de los modelos.

![imagen](img/download_train_spanish.png)

### Importamos librerias

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import seaborn as sns

from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

### Leemos el dataset

In [ ]:
#### CODE ####

In [ ]:
raw_dict = {
    'User': [],
    'Content': [],
    'Date': [],
    'Lang': [],
    'Polarity': [],
    'Type': []
}

for i in root.iter('tweet'):
    #### CODE ####
    
df = pd.DataFrame(raw_dict)
print(df.shape)

df.head()

### Columna de polaridad

In [ ]:
# Vemos los valores unicos de la columna de polaridad
#### CODE ####

In [ ]:
#### CODE ####

#### Columna de tipo

In [ ]:
#### CODE ####

## 2. Limpieza de datos
#### Polaridad
Vamos a clasificar los Tweets como buenos o malos, por lo que haremos la siguiente agrupación de la polaridad

In [ ]:
def polaridad_fun(x):
    if x in ('P', 'P+'):
        return 0
    elif x in ('N', 'N+'):
        return 1

In [ ]:
# Nos cargamos los NONE y los neutros
#### CODE ####
df['Polarity'].unique()

In [ ]:
# Pasamos la columna a 1s y 0s. Y el tipo
#### CODE ####
df['Polarity'].unique()

#### Idioma
Nos quedamos con los tweets en español. Si no tuviésemos esa columna podríamos acudir a librerías como `langid` o `langdetect`.

In [ ]:
# Filtramos los tweets en español
#### CODE ####

In [ ]:
# Vemos con cuantos registros nos hemos quedado despues del filtrado
df.shape

#### Duplicados

In [ ]:
# Eliminamos los duplicados
#### CODE ####
df.shape

#### Signos de puntuación
Eliminamos signos de puntuación: puntos, comas, interrogaciones, paréntesis

In [ ]:
df['Content'].head()

In [ ]:
import re

signos = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")

def signs_tweets(tweet):
    return signos.sub('', tweet.lower())

df['Content'] = df['Content'].apply(signs_tweets)
df['Content'].head()

#### Eliminamos links

In [ ]:
def remove_links(df):
    return " ".join(['{link}' if ('http') in word else word for word in df.split()])

df['Content'] = df['Content'].apply(remove_links)

#### Otros
Podríamos hacer un preprocesado mucho más fino:
1. Hashtags
2. Menciones
3. Abreviaturas
4. Faltas de ortografía
5. Risas

## 3. Modelo
Para montar el modelo tendremos que seguir los siguientes pasos
1. Eliminamos las stopwords
2. Aplicamos un stemmer, SnowBall por ejemplo

#### Stopwords

In [ ]:
#### CODE ####

def remove_stopwords(df):
    return " ".join([word for word in df.split() if word not in spanish_stopwords])

#### CODE ####
df.head()

#### Stemmer

In [ ]:
#### CODE ####

#### Seleccionamos columnas
Nos quedamos con las columnas que nos interesan para el modelo

In [ ]:
df = df[['Content', 'Polarity']]

#### Vectorizamos el dataset

In [ ]:
#### CODE ####

#### Montamos Pipeline
Modelos que suelen funcionar bien con pocas observaciones y muchas features son la Regresión logística el LinearSVC o Naive Bayes.

In [ ]:
#### CODE ####

# Aqui definimos el espacio de parámetros a explorar
parameters = {
    'vect__max_df': (0.5, 1.9),
    'vect__min_df': (10, 20,50),
    'vect__max_features': (500, 1000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigramas or bigramas
    'cls__C': (0.2, 0.5, 0.7),
    'cls__loss': ('hinge', 'squared_hinge'),
    'cls__max_iter': (500, 1000)
}


#### CODE ####

#### Entrenamos

In [ ]:
#### CODE ####

In [ ]:
print("Best params:", grid_search.best_params_)
print("Best acc:", grid_search.best_score_)
print("Best model:", grid_search.best_estimator_)

#### Guardamos el modelo

In [ ]:
import pickle

with open('finished_model.model', "wb") as archivo_salida:
    pickle.dump(grid_search.best_estimator_, archivo_salida)

## 4. Predicciones
#### API de Twitter
Lo primero que tenemos que hacer es conseguir nuevos Tweets. Para ello:
1. Nos registramos en la [web de desarrolladores de Twitter](https://developer.twitter.com/en/apply-for-access)
2. Bajamos el paquete `tweepy` para atacara  la API de Twitter
3. Buscamos un Hashtag de tendencia
4. Nos logamos y monitorizamos el hastag
5. Aplicamos la limpieza a los Tweets
6. Predecimos la polaridad

In [ ]:
# !pip install tweepy

In [ ]:
import tweepy  
import time
import csv

import json

with open('./credentials.json') as f:
    credentials = json.load(f)
    
    
# Credenciales de la web de desarroladores
access_token = credentials['access_token']  
access_token_secret = credentials['access_token_secret']  
consumer_key = credentials['consumer_key']
consumer_secret = credentials['consumer_secret'] 

# Nos autenticamos en la API
try:
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)  
    auth.set_access_token(access_token, access_token_secret)  
    api = tweepy.API(auth,wait_on_rate_limit=True)
    print("Authentication OK")
except:
    print("Error during authentication")

In [ ]:
raw_dict = {
    'author_name': [],
    'created_at': [],
    'content': [],
    'author_description': [],
    'author_followers_count': [],
    'author_profile_image_url': [],
    'author_location': [],
    'author_profile_background_image_url': [],
    'author_notifications': [],
    'geo': [],
    'coordinates': [],
    'entities': [],
    'place': []
}

# https://docs.tweepy.org/en/latest/api.html#search-methods

#### CODE ####

for tweet in cursor:
    
    raw_dict['author_name'].append(tweet.author.name)
    raw_dict['created_at'].append(tweet.created_at)
    raw_dict['content'].append(tweet.text)
    raw_dict['author_description'].append(tweet.author.description)
    raw_dict['author_followers_count'].append(tweet.author.followers_count)
    raw_dict['author_profile_image_url'].append(tweet.author.profile_image_url)
    raw_dict['author_location'].append(tweet.author.location)
    raw_dict['author_profile_background_image_url'].append(tweet.author.profile_background_image_url)
    raw_dict['author_notifications'].append(tweet.author.notifications)
    raw_dict['geo'].append(tweet.geo)
    raw_dict['coordinates'].append(tweet.coordinates)
    raw_dict['entities'].append(tweet.entities)
    raw_dict['place'].append(tweet.place)
    

test = pd.DataFrame(raw_dict)
test.head()

In [ ]:
print(test.shape)
print(test.author_profile_image_url[0])

#### Limpieza de test

In [ ]:
test_clean = test.copy()

# Seleccion de columnas
#### CODE ####

# Eliminar duplicados
#### CODE ####

# Signos de puntuacion
#### CODE ####

# Eliminamos links
#### CODE ####

# Nos cargamos stopwords
#### CODE ####

# Aplicamos el Stemmer
#### CODE ####
test_clean.shape

#### Leemos el pipeline con el modelo

In [ ]:
with open('finished_model.model', "rb") as archivo_entrada:
    pipeline_importada = pickle.load(archivo_entrada)
    
print(pipeline_importada)

#### Predicciones de test

In [ ]:
#### CODE ####

test_clean['Polarity'] = pd.Series(predictions)
for i in test_clean[:20].iterrows():
    print(i[1]['Polarity'])
    print(i[1]['content'])
    print('\n')